# How to ask questions to your PDF documents while your data stays on your computer.

In this notebook, we use a large language model to ask and answer questions to PDF documents. You may want to do this with non-publically available documents, or with documents that appeared after the language model was trained.

Additionally, we want to do run the language model locally. This has advantages for privacy and confidentiality (if you cannot send your data to a remote service).

The idea of how to ask questions to a document is to include the document in the prompt to the language model, so that it has additional context for answering your question. The prompt is limited by the length of the context window of the model, which may for example be 512 or 2048 for LLama 7B (measured in tokens). A single page can have 500 - 1000 tokens, so one needs to work around the limitation of a short context window. This can be done by 1) splitting your document(s) in reasonably sized pieces, and 2) only including the most relevant pieces in the prompt with your question.

This notebook therefore has the following structure:

* [**Load the model:**](#Load-the-model) We load the model.

* [**Read and split your documents:**](#Read-and-split-your-documents) We show how to read a PDF document (or several documents) and split them into pieces.

* [**Retrieve relevant pieces of your documents:**](#Retrieve-relevant-pieces-of-your-documents) We show how to create an embedding (= a low dimensional numerical representation) of your documents that helps us find the most relevant pieces of your documents. 

* [**Run the language model:**](#Run-the-language-model) We show how to include the relevant pieces in the question to the language model.

* [**Results**](#Results)


We will use Llama 7B as the language model, and [langchain](https://python.langchain.com/docs/get_started/introduction.html) for integration of the other necessary pieces. Langchain has many tutorials, the most similar to this write-up is the one on [Document QA](https://python.langchain.com/docs/modules/chains/additional/question_answering.html).

We will ask the question

In [1]:
query = "How many parameters does GPT-4 have?"

to the [GPT-4 Technical Report](https://arxiv.org/abs/2303.08774) (that does not contain the answer). 

In case you are only interested in the outcome: The results were pretty underwhelming. Technically, it works, but getting useful results seems to be a problem of finding the relevant information in your documents (search), and how to formulate the question to the language model (prompt engineering). Both problems are beyond the scope of this write-up.

## Load the model

We load the model. The first piece of code enables output of the model one token at a time:

In [2]:
# https://python.langchain.com/docs/modules/model_io/models/llms/integrations/llamacpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# https://python.langchain.com/docs/modules/model_io/models/llms/integrations/llamacpp
from langchain.llms import LlamaCpp
llm = LlamaCpp(
    model_path="models/llama/7B/ggml-model-q4_0.bin", 
    callback_manager=callback_manager, 
    verbose=True, 
    n_ctx=2048, 
    max_tokens=400
)

llama.cpp: loading model from models/llama/7B/ggml-model-q4_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.07 MB
llama_model_load_internal: mem required  = 5407.71 MB (+ 1026.00 MB per state)
llama_new_context_with_model: kv self size  = 1024.00 MB
AVX = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 0 | NEON = 1 | ARM_FMA = 1 | F16C = 0 | FP16_VA = 1 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 0 | VSX = 0 | 


To use Llama, you need to follow the steps from [llama.cpp](https://github.com/ggerganov/llama.cpp) first. This requires you to officially [apply for access](https://ai.facebook.com/blog/large-language-model-llama-meta-ai/) to the Llama model. Alternatively, langchain supports many other language models, see "Integrations" at [langchain LLMs](https://python.langchain.com/docs/modules/model_io/models/llms).

Note: Most currently available models are not commercially useable. At time of writing, [MPT-7B](https://www.mosaicml.com/blog/mpt-7b) is an exception, but  does not seem to be fully integrated in langchain yet (see [MosaicML](https://python.langchain.com/docs/modules/model_io/models/llms/integrations/mosaicml)).

(The arguments `n_ctx` for the length of the context window, and `max_tokens` to restrict the length of the output produced by the model will become important later. For reference see all possible arguments [here](https://api.python.langchain.com/en/latest/llms/langchain.llms.llamacpp.LlamaCpp.html?highlight=llamacpp#langchain.llms.llamacpp.LlamaCpp).)

## Read and split your documents

The following code loads a single PDF.

In [3]:
# from https://python.langchain.com/docs/modules/data_connection/document_loaders/how_to/pdf
# and https://www.anyscale.com/blog/turbocharge-langchain-now-guide-to-20x-faster-embedding
# requires package pypdf
from langchain.document_loaders import PyPDFLoader
doc = PyPDFLoader("gpt-4.pdf").load()

The list `doc` contains a representation for each page.

To load an entire directory, you can use 

    from langchain.document_loaders import PyPDFDirectoryLoader
    docs = PyPDFDirectoryLoader("pdf_directory/").load()

The following code splits the document (pages) in pieces that fit in the context window of the model:

In [4]:
# from https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=480,
    chunk_overlap=80,
    length_function=llm.get_num_tokens
)
split_docs = text_splitter.split_documents(doc)

The documents (= pages of the PDF) are split into pieces of length at most equal to `chunk_size`, while paragraphs, sentences, and words are respected (not all elements of `split_docs` will have length equal to `chunk_size`). If `chunk_overlap` is a positive number, the individual pieces of text can overlap (but don't have to, if sentences or paragraphs cannot be respected). [This interactive website](https://belladoreai.github.io/llama-tokenizer-js/example-demo/build/) gives an idea how many tokens a given text has.

## Retrieve relevant pieces of your documents 

Here we create an "embedding" of the pieces of our document. Embeddings are a function that map a given text to a vector of numbers. The numeric representation has the additional property that texts that are 'similar' to each other also have vectors that are 'close' to each other.

We will use this approach to find parts of the document that may be relevant to the question.

In [5]:
# from https://python.langchain.com/docs/modules/data_connection/text_embedding/integrations/sentence_transformers
# requires package sentence_transformers
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

The class SentenceTransformerEmbeddings from langchain is essentially a wrapper around the package [sentence_transformers](https://www.sbert.net/examples/applications/computing-embeddings/README.html?highlight=sentencetransformer#sentence_transformers.SentenceTransformer).
The argument `model_name` lets you choose the embedding model, see the [list of supported models](https://www.sbert.net/docs/pretrained_models.html).

In case of the model `all-MiniLM-L6-v2`, the pieces of our document will be encoded as vectors of length 384.

The following code creates an embedding of all pieces of text and saves them in a special object ("vector store") along with the input texts, and some additional information (such as the page number of the input PDF):

In [6]:
# requires package chromadb
from langchain.vectorstores import Chroma
docsearch = Chroma.from_documents(split_docs, embeddings)

We retrieve the `k = 4` most relevant pieces of text to include in the question prompt:

In [7]:
# see also API reference to similarity_search: https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.chroma.Chroma.html#langchain.vectorstores.chroma.Chroma.similarity_search
docs = docsearch.similarity_search(query=query, k=4)

## Run the language model

There are several ways how to include the relevant pieces of text to the language model (the parameter `chain_type` in the following code snippet):

* [stuff](https://python.langchain.com/docs/modules/chains/document/stuff): All relevant pieces of text are included together with the question in the prompt to the language model.

* [refine](https://python.langchain.com/docs/modules/chains/document/refine): The model is run once for each piece of relevant text, each time with the piece of text and the previous answer. It is asked to improve the answer, if possible, given the additional information.

* [map_reduce](https://python.langchain.com/docs/modules/chains/document/map_reduce): The model is asked to produce an answer given one piece of text, once for all the relevant pieces. Given all the individual answers, then the model is then asked to produce a final answer.

* [map_rerank](https://python.langchain.com/docs/modules/chains/document/map_rerank): The model is run once for each piece of text and produces an answer and a score how certain it is. The answer with the highest score is the final answer.

Because the context window (length of the prompt) of a language model is finite, each of the method places constraints on the length of the individual pieces of text, and/or on the model output (measured in tokens).

For example, for `chain_type = stuff`, the following are part of the prompt:
* Your question,
* all pieces of relevant text,
* some additional system prompt to explain to the model what to do.

In our setting, the context window has length `n_ctx=2048`and we retrieve `k=4` most relevant pieces of text. Together with the system prompt and our question, this places the constraint of at most <512 tokens to the length of each piece of text (we chose `chunk_size=480`).

The system prompts are given by langchain. For example the [system prompt for the 'stuff' method](https://github.com/hwchase17/langchain/blob/master/langchain/chains/question_answering/stuff_prompt.py) is "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

In [8]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm, chain_type="stuff")
chain.run(input_documents=docs, question=query, return_only_outputs=True)

 We don’t know exactly how many parameters it has.
60

10324

970

GPT-4 was trained on 35TB of text, which is over a million times more than the size of TD-VL, and about three orders of magnitude larger than GPT-3.
The model has four times the number of parameters as GPT-3.5
(Table 10).
75

We use four variants: GPT-4-base for zero-shot prompting; GPT-4-turbo for few shot prompting; and GPT-4-launch, which is after a RLHF-based optimization of GPT-4-base.
The difference between base GPT-3 and Turbo GPT-3 is around 10−5%. The difference between base GPT-3 and Turbo GPT-4 is roughly 10−7% (Table 15).
Table 2: Model parameters for each variant of GPT-4.
The number of tokens we use is 68 or 69, depending on the model version.
We find that the accuracy across variants is similar (Table 3), suggesting that a lot of
the performance comes from simply having a larger training corpus with more tokens.
We also observe that GPT-4’s performance on zero-shot prompting and few-shot prompting are sim


llama_print_timings:        load time =   485.10 ms
llama_print_timings:      sample time =   316.15 ms /   400 runs   (    0.79 ms per token,  1265.23 tokens per second)
llama_print_timings: prompt eval time = 23402.58 ms /   456 tokens (   51.32 ms per token,    19.49 tokens per second)
llama_print_timings:        eval time = 30726.68 ms /   399 runs   (   77.01 ms per token,    12.99 tokens per second)
llama_print_timings:       total time = 55281.87 ms


' We don’t know exactly how many parameters it has.\n60\n\n10324\n\n970\n\nGPT-4 was trained on 35TB of text, which is over a million times more than the size of TD-VL, and about three orders of magnitude larger than GPT-3.\nThe model has four times the number of parameters as GPT-3.5\n(Table 10).\n75\n\nWe use four variants: GPT-4-base for zero-shot prompting; GPT-4-turbo for few shot prompting; and GPT-4-launch, which is after a RLHF-based optimization of GPT-4-base.\nThe difference between base GPT-3 and Turbo GPT-3 is around 10−5%. The difference between base GPT-3 and Turbo GPT-4 is roughly 10−7% (Table 15).\nTable 2: Model parameters for each variant of GPT-4.\nThe number of tokens we use is 68 or 69, depending on the model version.\nWe find that the accuracy across variants is similar (Table 3), suggesting that a lot of\nthe performance comes from simply having a larger training corpus with more tokens.\nWe also observe that GPT-4’s performance on zero-shot prompting and few-sho

## Results



Despite trying many different variations, the model never said 'I don't know.', or 'Based on the information available to me, I cannot give an answer.'.

Additionally, the model produces quite a lot of output that does not seem to be an answer to the question. This seems to be connected to the way the question and system prompt are phrased, see for example [this discussion](https://github.com/facebookresearch/llama/issues/69) (in general, this seems to amount to the difference between language model and chat model).

When running the same question as in the langchain tutorial on [Document QA](https://python.langchain.com/docs/modules/chains/additional/question_answering), "What did the president say about Justice Breyer" on the given example document, answers are produced that are reasonably helpful, but you would get the same answer just by searching for "Justice Breyer" in the provided document.

Finally, GPT-3.5-turbo gives a correct answer using this code to replace the model:

In [19]:
# requires package openai and your Open AI API key
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(openai_api_key=key, model="gpt-3.5-turbo")
chain = load_qa_chain(llm, chain_type="stuff")
chain.run(input_documents=docs, question=query, return_only_outputs=True)

"The given context does not mention the number of parameters for GPT-4. Therefore, I don't have the information to answer your question."

"The given context does not provide information about the number of parameters in GPT-4. Therefore, I don't know the exact number of parameters in GPT-4."

At this point, the tutorial stops, and the following topics are left for further exploration:

* **Document storage and retrival** (embeddings and similarity search): Finding the pieces of your documents that are most relevant to answer your question.
* **System prompt design** (promt engineering): Nudging the language model towards producing useful answers.
* **Use larger models locally**: Presumably this would also increase performance.